## 10. 스태킹 앙상블
- 개별적인 여러 알고리즘을 서로 결합해 예측 결과를 도출 (배깅 및 부스팅과 공통점)
- 개별 알고리즘으로 예측한 데이터를 기반으로 다시 예측을 수행 (나머지 앙상블 학습방식과 차이점)   
cf. 부스팅 알고리즘은 잘못 예측한 데이터에 가중치를 부여하며 오류를 개선
- 개별 알고리즘의 예측 결과 데이터 세트를 최종적인 <U>메타 데이터 세트</U>로 만들어, 별도의 ML 알고리즘으로 최종 학습을 수행하고 테스트 데이터를 기반으로 다시 최종 예측을 수행하는 방식
- 이 때 개별 모델의 예측된 데이터 세트를 기반으로 하여 학습하고 예측하는 방식을 <U>메타 모델</U>이라고 함

<img src = "https://media.vlpt.us/images/dbj2000/post/8cf8374b-4ea6-437d-a0b3-92276af08abc/image.png" width="100%">

- 필요한 모델 1) 개별적인 기반 모델 2) 최종 메타 모델
- 핵심 : 여러 개별 모델의 예측 데이터를 스태킹 형태로 결합해 최종 메타 모델의 학습용 및 테스트용 데이터 세트를 만드는 것
- 현실 모델 X, 캐글 같은 대회 O, 일반적으로 성능이 비슷한 모델을 결합

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### 기본 스태킹 모델  : 위스콘신 암 데이터 세트
- 30개의 피처 → 악성(1)인지 양성(0)인지
- 개별 모델 : KNN, 랜덤포레스트, 결정트리, 에이다부스트
- 최종 메타 모델 : 로지스틱 회귀

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer = load_breast_cancer()
X_data = cancer.data
y_label = cancer.target
print(X_data.shape, y_label.shape)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, stratify=y_label, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape)

(569, 30) (569,)
(455, 30) (114, 30)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
tree_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

lr_final = LogisticRegression(C=10) # C : inverse of regularization strength; = SVM

knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
tree_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
tree_pred = tree_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print("========== ACCURACY ==========")
print("knn : {:.4f}".format( accuracy_score(y_test, knn_pred)))
print("randomforest : {:.4f}".format(accuracy_score(y_test, rf_pred)))
print("decisiontree :{:.4f} ".format(accuracy_score(y_test, tree_pred)))
print("adaboost : {:.4f}".format(accuracy_score(y_test, ada_pred)))


========== ACCURACY ==========
knn : 0.9035
randomforest : 0.9474
decisiontree :0.9123 
adaboost : 0.9474


In [9]:
pred = np.array([knn_pred, rf_pred, tree_pred, ada_pred])
print(pred.shape)
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [10]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)
print("========== ACCURACY ==========")
print("logistic (meta model) : {:.4f}".format(accuracy_score(y_test, final)))

========== ACCURACY ==========
logistic (meta model) : 0.9649


#### CV세트 기반의 스태킹
- 위의 예제에서 로지스틱 회귀 모델을 학습할 때 y_train이 아닌 y_test로 학습했기에 과적합 문제가 발생할 수 있음   
→ CV 세트 기반의 스태킹에서 개별 모델들이 각각 교차 검증으로 메타 모델을 위한 학습용 및 테스트용 스태킹 데이터를 생성한 뒤 이를 기반으로 메타 모델이 학습과 예측을 수행

1. 각 모델별로 원본 학습용/테스트용 데이터를 예측한 결과 값을 기반으로 메타 모델을 위한 학습용/테스트용 데이터를 생성
2. 1.에서 생성한 학습용 데이터를 모두 스태킹 형태로 합쳐 메타 모델이 학습할 최종 학습용 데이터 세트를 생성
3. 마찬가지로 각 모델들이 생성한 테스트용 데이터를 모두 스태킹으로 합쳐서 메타 모델이 최종적으로 학습할 데이터를 생성
4. 메타 모델은 최종적으로 생성된 학습 데이터와 원본 레이블 데이터를 기반으로 학습한 뒤, 최종적으로 생성된 테스트 데이터 세트를 예측하고, 원본 레이블 데이터를 기반으로 평가

<img src ="https://media.vlpt.us/images/dbj2000/post/1d9611a2-2ca4-4516-8ffe-562e044d5b47/image.png" width="100%">

<img src = "https://media.vlpt.us/images/dbj2000/post/73814738-3d92-4412-9f5e-36e73ca04222/image.png" width="100%">

<img src = "https://media.vlpt.us/images/dbj2000/post/5a65a534-ee72-48dc-9efb-f317d4ebf9c6/image.png" width="100%">

<img src="https://media.vlpt.us/images/sset2323/post/364876bb-9781-4d62-9076-84b16bec6762/image.png" width="100%">

In [16]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [23]:
from sklearn.model_selection import KFold

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0) # 분류이기 때문에 shuffle=True

    train_fold_pred = np.zeros((X_train_n.shape[0],1))
    test_fold_pred = np.zeros((X_test_n.shape[0],n_folds)) 
    print(model.__class__.__name__)
    
    for fold_counter, (train_idx, valid_idx) in enumerate(kf.split(X_train_n)) :
        print('{} 폴드 세트'.format(fold_counter))
        X_tr = X_train_n[train_idx, :]
        y_tr = y_train_n[train_idx]
        X_val = X_train_n[valid_idx, :]
        y_val = y_train_n[valid_idx]

        # 학습
        model.fit(X_tr, y_tr)
        # 예측
        train_pred = model.predict(X_val).reshape(-1,1)
        test_pred = model.predict(X_test_n)
        # 저장
        train_fold_pred[valid_idx, :] = train_pred
        test_fold_pred[:, fold_counter] = test_pred

    test_fold_pred_mean = np.mean(test_fold_pred, axis=1).reshape(-1,1)

    print(train_fold_pred.shape, test_fold_pred_mean.shape)
    return train_fold_pred, test_fold_pred_mean


In [27]:
knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
tree_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

n_folds=7
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, n_folds)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, n_folds)
tree_train, tree_test = get_stacking_base_datasets(tree_clf, X_train, y_train, X_test, n_folds)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, n_folds)

KNeighborsClassifier
0 폴드 세트
1 폴드 세트
2 폴드 세트
3 폴드 세트
4 폴드 세트
5 폴드 세트
6 폴드 세트
(455, 1) (114, 1)
RandomForestClassifier
0 폴드 세트
1 폴드 세트
2 폴드 세트
3 폴드 세트
4 폴드 세트
5 폴드 세트
6 폴드 세트
(455, 1) (114, 1)
DecisionTreeClassifier
0 폴드 세트
1 폴드 세트
2 폴드 세트
3 폴드 세트
4 폴드 세트
5 폴드 세트
6 폴드 세트
(455, 1) (114, 1)
AdaBoostClassifier
0 폴드 세트
1 폴드 세트
2 폴드 세트
3 폴드 세트
4 폴드 세트
5 폴드 세트
6 폴드 세트
(455, 1) (114, 1)


In [26]:
stack_X_train = np.concatenate([knn_train, rf_train, tree_train, ada_train], axis=1)
stack_X_test = np.concatenate([knn_test, rf_test, tree_test, ada_test], axis=1)

print(stack_X_train.shape, stack_X_test.shape)

(455, 4) (114, 4)


In [29]:
lr_clf = LogisticRegression(C=10)
lr_clf.fit(stack_X_train,y_train)
stack_final = lr_clf.predict(stack_X_test)

print(stack_final.shape)
print("stacking acc : {:.4f}".format(accuracy_score(y_test, stack_final)))

(114,)
stacking acc : 0.9737


#### 스태킹 모델의 하이퍼파라미터 튜닝
- 일반적으로 개별 알고리즘 모델의 파라미터를 최적으로 튜닝하는 것을 말함


In [38]:
print(np.unique(stack_X_train, return_counts=True))
print(np.unique(stack_X_test, return_counts=True))


(array([0., 1.]), array([ 683, 1137]))
(array([0.        , 0.14285714, 0.28571429, 0.42857143, 0.57142857,
       0.71428571, 0.85714286, 1.        ]), array([153,  13,   6,   2,   6,   3,   8, 265]))
